<a href="https://colab.research.google.com/github/cosineDaDa0623/2023_Summer_Intern/blob/master/Pytorch_dc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install rdkit
!pip install deepchem[torch]
!pip install torch_geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.7/29.7 MB 49.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 693.2/693.2 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 MB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 725.0/725.0 kB 43.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 68.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.1/226.1 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.6/731.6 kB 48.5 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.10.1
    Uninstalling scipy-1.10.1:
      Successfully uninstalled scipy-1.10.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 661.6/661.6 kB 6.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch_geometric: filename=torch_geometric-2.3.1-

In [2]:
import deepchem as dc
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit import DataStructs
import pandas as pd #Pandas have Dataframe, so the manipulation of data is efficient and convenient
import numpy as np #Linear math packages, which are important for the data
from google.colab import drive #Just used to get file from the google drive
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.utils import data
from torch import nn
drive.mount('/content/drive')
# df = pd.read_excel('/content/drive/MyDrive/5HT2A models data.xlsx')
from sklearn.metrics import precision_score, recall_score, f1_score, matthews_corrcoef

Mounted at /content/drive


In [ ]:
# def get_response(ligand_class):
#   convert_dic = {
#       'dude_active' : 0,
#       'dude_inactive' : 1,
#       'HTR2A_Antagonist_1.0' : 2,
#       'HTR2A_Antagonist_0.0' : 3,
#       'HTR2A_Agonist_1.0' : 4,
#       'HTR2A_Agonist_0.0' : 5,
#       'HTR2A_Inverse Agonist_1.0' : 6
# }
#   return convert_dic[ligand_class]

# df['label'] = df['ligand_class'].apply(get_response)

In [ ]:
# df['label'].unique()

array([0, 1, 2, 4, 5, 3, 6])

In [ ]:
# df.to_csv('/content/drive/MyDrive/5HT2A models data.csv')

In [81]:
dataset_file = '/content/drive/MyDrive/5HT2A models data.csv'
task_list = ['label']
featurizer_func = dc.feat.CircularFingerprint(size = 2048, radius = 4)

In [82]:
loader = dc.data.CSVLoader(tasks=task_list, feature_field='smiles', featurizer=featurizer_func)
dataset = loader.create_dataset(dataset_file)

In [83]:
scaffoldsplitter = dc.splits.ScaffoldSplitter()
test_dataset, vadid_dataset, test_dataset = scaffoldsplitter.train_valid_test_split(dataset, frac_train = 0.7, frac_valid = 0.15, frac_test = 0.15, seed = 23)

In [84]:
tt = test_dataset.make_pytorch_dataset(batch_size = 32)
va = vadid_dataset.make_pytorch_dataset(batch_size = 32)
te = test_dataset.make_pytorch_dataset(batch_size = 32)

In [89]:
#Import Pytorch which is a commonly-used deeplearning package

class NN1(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear_layers = nn.Sequential(
        nn.Linear(2048, 1024),
        nn.BatchNorm1d(1024),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(1024, 512),
        nn.BatchNorm1d(512),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(512, 256),
        nn.BatchNorm1d(256),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(256, 7)
    )

  def forward(self, x):
    return self.linear_layers(x)

In [90]:
def train_loop(dataloader, model, loss_fn, optimizer):
    i = 0
    # Set the model to training mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.train()
    for batch, (X, y, _, _) in enumerate(dataloader):
      X = torch.tensor(X).float()
      y = torch.tensor(y).long()
      y = y.squeeze(1)
      pred = model(X)
      loss = loss_fn(pred, y)
      # Backpropagation
      loss.backward()
      optimizer.step()
      optimizer.zero_grad()
      if batch % 100 == 0:
          loss, current = loss.item(), (batch + 1) * len(X)
          print(f"loss: {loss:>7f}")

def test_loop(dataloader, model, loss_fn):
  model.eval()
  size = len(dataloader.disk_dataset)
  num_batches = 0
  test_loss, correct = 0, 0

  # Keep track of true and predicted labels for additional metrics
  all_true_labels = []
  all_predicted_labels = []

  with torch.no_grad():
      for (X, y, _, _) in dataloader:
          X = torch.tensor(X).float()
          y = torch.tensor(y).long()
          y = y.squeeze(1)
          pred = model(X)
          test_loss += loss_fn(pred, y).item()
          correct += (pred.argmax(1) == y).type(torch.float).sum().item()

          # Add true labels and predicted labels to lists
          all_true_labels.extend(y.tolist())
          all_predicted_labels.extend(pred.argmax(1).tolist())
          num_batches += 1

  test_loss /= num_batches
  correct /= size

  # Compute additional metrics
  precision = precision_score(all_true_labels, all_predicted_labels, average='macro')
  recall = recall_score(all_true_labels, all_predicted_labels, average='macro')
  f1 = f1_score(all_true_labels, all_predicted_labels, average='macro')
  mcc = matthews_corrcoef(all_true_labels, all_predicted_labels)

  print(f"Test Error: ")
  print(f" Accuracy: {(100*correct):>0.1f}%")
  print(f" Avg loss: {test_loss:>8f}")
  print(f" Precision: {precision:>0.4f}")
  print(f" Recall: {recall:>0.4f}")
  print(f" F1 Score: {f1:>0.4f}")
  print(f" Matthew's Correlation Coefficient: {mcc:>0.4f}")
  return test_loss

In [97]:
class EarlyStopping:
    def __init__(self, patience=10):
        self.patience = patience
        self.counter = 0
        self.best_score = None
        self.early_stop = False

    def __call__(self, test_loss, model):

        if self.best_score is None:
            self.best_score = test_loss
            self.save_checkpoint(model)
        elif test_loss >= self.best_score:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = test_loss
            self.save_checkpoint(model)
            self.counter = 0

    def save_checkpoint(self, model):
        torch.save(model.state_dict(), '/content/drive/MyDrive/checkpoint.pt')

In [99]:
nn1 = NN1()
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(nn1.parameters(), lr=0.01)
early_stopping = EarlyStopping(patience=10)
epochs = 100
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(tt, nn1, loss_fn, optimizer)
    test_loss = test_loop(va, nn1, loss_fn)
    early_stopping(test_loss, nn1)
    if early_stopping.early_stop:
        print("Early stopping")
        break
print("Done!")

Epoch 1
-------------------------------
loss: 1.837479
loss: 1.024665


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Test Error: 
 Accuracy: 69.6%
 Avg loss: 1.148619
 Precision: 0.3410
 Recall: 0.3534
 F1 Score: 0.3463
 Matthew's Correlation Coefficient: 0.5031
Epoch 2
-------------------------------
loss: 0.889622
loss: 0.696316


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Test Error: 
 Accuracy: 71.8%
 Avg loss: 0.952582
 Precision: 0.3503
 Recall: 0.3807
 F1 Score: 0.3485
 Matthew's Correlation Coefficient: 0.5714
Epoch 3
-------------------------------
loss: 0.970917
loss: 0.562620


...

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Test Error: 
 Accuracy: 74.2%
 Avg loss: 0.900651
 Precision: 0.3995
 Recall: 0.4611
 F1 Score: 0.4049
 Matthew's Correlation Coefficient: 0.6260
EarlyStopping counter: 8 out of 10
Epoch 16
-------------------------------
loss: 0.120204
loss: 0.104190


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Test Error: 
 Accuracy: 73.2%
 Avg loss: 0.958873
 Precision: 0.3979
 Recall: 0.4567
 F1 Score: 0.3978
 Matthew's Correlation Coefficient: 0.6128
EarlyStopping counter: 9 out of 10
Epoch 17
-------------------------------
loss: 0.453624
loss: 0.142173
Test Error: 
 Accuracy: 72.9%
 Avg loss: 0.987175
 Precision: 0.3961
 Recall: 0.4525
 F1 Score: 0.3955
 Matthew's Correlation Coefficient: 0.6114
EarlyStopping counter: 10 out of 10
Early stopping
Done!


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [101]:
model = NN1()
model.load_state_dict(torch.load("/content/drive/MyDrive/checkpoint.pt"))
test_loop(te, nn1, loss_fn)

Test Error: 
 Accuracy: 98.4%
 Avg loss: 0.050504
 Precision: 0.9031
 Recall: 0.6789
 F1 Score: 0.6984
 Matthew's Correlation Coefficient: 0.9754


0.05050356642813117